In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64 # for images
import re # needed for regex

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "myAACuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style = {'width': '200px', 'height': '200px'})),
    html.Center(html.B(html.H4('Austin Animal Center Dashboard'))),
    html.Center(html.B(html.H5('Breanna Hodges - SNHU CS-340'))),
    html.Hr(),
    html.Div(    
        # radio buttons to filter data
        dcc.Dropdown(
            id = 'filter-type',
            options = [
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'RESET'}
            ],
            value = 'RESET',
        )
    ),
    
    html.Hr(),
    dt.DataTable(
        id = 'datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=15,
           
    ),
    
    html.Br(),
    html.Hr(),
# sets up dashboard so chart and geolocation are side by side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[ 
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ]),
    html.Div(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style = {'width': '100px', 'height': '100px'})),
    html.Div(html.A("Created By: Breanna Hodges"))

])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type','value')])
def update_dashboard(filter_type):
### if/else block for drop down
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
    
    elif filter_type == 'Water':
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or': [
                {"breed": {'$regex': newRegex}},
                {'breed': {'$regex': chesaRegex}},
                {'breed': {'$regex': labRegex}},
            ],
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }))
        
    elif filter_type == 'Mountain':
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old english.*", re.IGNORECASE)
        siberianRegex = re.compile(".*siberian.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or': [
                {"breed": {'$regex': germanRegex}},
                {'breed': {'$regex': alaskanRegex}},
                {'breed': {'$regex': oldRegex}},
                {'breed': {'$regex': siberianRegex}},
                {'breed': {'$regex': rottRegex}},
            ],
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }))
        
        
    elif filter_type == 'Disaster':
        dobermanRegex = re.compile(".*doberman.*", re.IGNORECASE)
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*bloodhound.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or': [
                {"breed": {'$regex': dobermanRegex}},
                {'breed': {'$regex': germanRegex}},
                {'breed': {'$regex': goldenRegex}},
                {'breed': {'$regex': bloodRegex}},
                {'breed': {'$regex': rottRegex}},
            ],
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20.0, '$lte': 300.0}
        }))
    else:
        raise Exception("Unknown filter")
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)



@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
# change color of selected cell
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #pie chart for the different breeds
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame = dff,
                values = values,
                names = names,
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800,
                height = 500
            )
        )
    ]

                                
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, 
           center=[30.75,-97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup shows animal name, sex, breed, and age
           dl.Marker(position=[30.75,-97.48], children=[dl.Tooltip(dff.iloc[0,14]),
           dl.Popup([html.H4('Animal Name'), html.P(dff.iloc[0, 9]),
                     html.H4("Sex"), html.P(dff.iloc[0, 12]),
                     html.H4("Breed"), html.P(dff.iloc[0, 4]),
                     html.H4("Age"), html.P(dff.iloc[0, 15])
                     ])
                ])
          ])
    ]




app